# VacationPy
----
### Observations 
After gathering and analyzing api data on weather conditions: temperture, humidity, cloud cover, wind speed, and geolocation (latitudinal distance from the equator) for 565 cities in both the northern (384) and southern (181) hemisphere for the month of Febuary 2022, there seems to be a negative correlation between temperature (in degrees Fahrenheit) and geolocation from the equator for cities in the northern hemisphere and a positive correlation for the same metrics for cities located in the southern hemisphere. The farther away a city is located from the equator (0 degrees latitude) the colder the climate. 





In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# import csv from WeatherPy 
weather_csv = "../WeatherPy/output_data/cities.csv"

weather_df = pd.read_csv(weather_csv)

weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Portland,45.5234,-122.6762,50.79,87,75,1.99,US,1644427595
1,1,Mahébourg,-20.4081,57.7000,79.09,78,40,3.44,MU,1644427732
2,2,Adrar,20.5022,-10.0711,83.23,6,19,16.20,MR,1644427733
3,3,Atuona,-9.8000,-139.0333,78.06,77,24,18.66,PF,1644427578
4,4,Saint-Philippe,-21.3585,55.7679,78.39,79,98,8.39,RE,1644427733
...,...,...,...,...,...,...,...,...,...,...
564,564,Bosobolo,4.1833,19.9000,83.19,20,47,1.88,CD,1644427902
565,565,Wanaka,-44.7000,169.1500,63.30,95,99,3.18,NZ,1644427902
566,566,Santa Maria,-29.6842,-53.8069,81.70,39,20,8.05,BR,1644427903
567,567,Bhinga,27.7167,81.9333,55.67,86,49,5.19,IN,1644427903


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Config gmaps with api key.
gmaps.configure(api_key=g_key)

# Drop NA
weather_df = weather_df.dropna()

# Convert to Float
humidity = weather_df["Humidity"].astype(float)

# Storage 
location = weather_df[["Lat", "Lng"]].astype(float)

# Humidity Heatmap Layer
figure = gmaps.figure()

heat = gmaps.heatmap_layer(location, weights = humidity, dissipating = False, max_intensity = max(weather_df["Humidity"]), point_radius = 2)

figure.add_layer(heat)

figure

Figure(layout=FigureLayout(height='420px'))

In [4]:
# New DataFrame
ideal_weather_df = weather_df

"""
Conditionals: 

A max temperature lower than 80 degrees but higher than 70.
Wind speed less than 10 mph.
Zero cloudiness.

Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
"""
# A max temperature lower than 80 degrees but higher than 70.
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp"] < 80) & (ideal_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] == 0]

# Drop NA
ideal_weather_df = ideal_weather_df.dropna()

ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
143,143,Ormara,25.2088,64.6357,71.24,67,0,3.62,PK,1644427773
148,148,Salalah,17.0151,54.0924,75.29,12,0,6.91,OM,1644427586
196,196,Cabo San Lucas,22.8909,-109.9124,76.05,56,0,3.44,MX,1644427790
255,255,Saint-Pierre,-21.3393,55.4781,78.48,65,0,6.91,RE,1644427812
288,288,Makakilo City,21.3469,-158.0858,71.92,81,0,4.61,US,1644427644
290,290,Lompoc,34.6391,-120.4579,76.57,43,0,6.91,US,1644427549
421,421,Bucerías,20.7500,-105.3333,73.38,64,0,2.30,MX,1644427860
501,501,Aswān,24.0934,32.9070,72.70,13,0,6.91,EG,1644427883


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_map_df = ideal_weather_df

# params, iteration, etc.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# iteration 
for index, row in hotel_map_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat}, {lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    print(f"Result for the Index {index}: {row['City']}.")
    
    # response
    response = requests.get(url, params=params).json()
    
    results = response["results"]
    
    try:
        print(f"Nearest hotel is {results[0]['name']}.")
        hotel_map_df.loc[index, "Name of Hotel"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Skip Over Missing Info...")
        
    # print delimiter 
    print("---------------------------------")
# 10 results 

Result for the Index 143: Ormara.
Nearest hotel is Crystal Beach and Camping Resort Ormara.
---------------------------------
Result for the Index 148: Salalah.
Nearest hotel is HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel.
---------------------------------
Result for the Index 196: Cabo San Lucas.
Nearest hotel is Hotel Tesoro Los Cabos.
---------------------------------
Result for the Index 255: Saint-Pierre.
Nearest hotel is Lindsey Hôtel.
---------------------------------
Result for the Index 288: Makakilo City.
Nearest hotel is Marriott's Ko Olina Beach Club.
---------------------------------
Result for the Index 290: Lompoc.
Nearest hotel is Holiday Inn Express Lompoc, an IHG Hotel.
---------------------------------
Result for the Index 421: Bucerías.
Nearest hotel is Royal Decameron Complex - Puerto Vallarta.
---------------------------------
Result for the Index 501: Aswān.
Nearest hotel is Nuba Nile.
---------------------------------


In [6]:
hotel_map_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Name of Hotel
143,143,Ormara,25.2088,64.6357,71.24,67,0,3.62,PK,1644427773,Crystal Beach and Camping Resort Ormara
148,148,Salalah,17.0151,54.0924,75.29,12,0,6.91,OM,1644427586,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
196,196,Cabo San Lucas,22.8909,-109.9124,76.05,56,0,3.44,MX,1644427790,Hotel Tesoro Los Cabos
255,255,Saint-Pierre,-21.3393,55.4781,78.48,65,0,6.91,RE,1644427812,Lindsey Hôtel
288,288,Makakilo City,21.3469,-158.0858,71.92,81,0,4.61,US,1644427644,Marriott's Ko Olina Beach Club
290,290,Lompoc,34.6391,-120.4579,76.57,43,0,6.91,US,1644427549,"Holiday Inn Express Lompoc, an IHG Hotel"
421,421,Bucerías,20.7500,-105.3333,73.38,64,0,2.30,MX,1644427860,Royal Decameron Complex - Puerto Vallarta
501,501,Aswān,24.0934,32.9070,72.70,13,0,6.91,EG,1644427883,Nuba Nile


In [7]:
# Add marker layer ontop of heat map
locations = hotel_map_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name of Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_map_df.iterrows()]
locations = hotel_map_df[["Lat", "Lng"]]

# Symbol Layer
hotel_map_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)', stroke_color='rgba(0, 0, 150, 0.4)', scale=2, info_box_content=hotel_info)

# Additional Layers
figure.add_layer(markers)
figure.add_layer(hotel_map_layer)

# Display figure

figure

Figure(layout=FigureLayout(height='420px'))